## 2. Sentiment Analysis
In this exercise, we will classify the sentiment of text documents. Complete the code with TODO tag.

References and Further Readings:
+ http://www.nltk.org/howto/sentiment.html
+ https://www.nltk.org/api/nltk.sentiment.html
+ http://datameetsmedia.com/vader-sentiment-analysis-explained/
+ https://github.com/cjhutto/vaderSentiment
+ https://marcobonzanini.com/2015/05/17/mining-twitter-data-with-python-part-6-sentiment-analysis-basics/
+ https://github.com/marrrcin/ml-twitter-sentiment-analysis


### 2.1. Classification approach

Classification approach looks at previously labeled data in order to determine the sentiment of never-before-seen sentences. It involves training a model using previously seen text to predict/classify the sentiment of some new input text. The nice thing is that, with a greater volume of data, we generally get better prediction or classification results. However, unlike the lexical approach, we need previously labeled data.

In [2]:
import nltk
from nltk.classify import NaiveBayesClassifier
# nltk.download('subjectivity')
nltk.download('movie_reviews')
from nltk.corpus import subjectivity
from nltk.corpus import movie_reviews
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

# n_instances = 100
# subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
# obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
# len(subj_docs), len(obj_docs)

n_instances = None
if n_instances is not None:
    n_instances = int(n_instances/2)

pos_docs = [(list(movie_reviews.words(pos_id)), 'pos') for pos_id in movie_reviews.fileids('pos')[:n_instances]]
neg_docs = [(list(movie_reviews.words(neg_id)), 'neg') for neg_id in movie_reviews.fileids('neg')[:n_instances]]
len(pos_docs), len(neg_docs)

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


(1000, 1000)

Each document is represented by a tuple (sentence, label). The sentence is tokenized, so it is represented by a list of strings:

In [3]:
pos_docs[0]

(['films',
  'adapted',
  'from',
  'comic',
  'books',
  'have',
  'had',
  'plenty',
  'of',
  'success',
  ',',
  'whether',
  'they',
  "'",
  're',
  'about',
  'superheroes',
  '(',
  'batman',
  ',',
  'superman',
  ',',
  'spawn',
  ')',
  ',',
  'or',
  'geared',
  'toward',
  'kids',
  '(',
  'casper',
  ')',
  'or',
  'the',
  'arthouse',
  'crowd',
  '(',
  'ghost',
  'world',
  ')',
  ',',
  'but',
  'there',
  "'",
  's',
  'never',
  'really',
  'been',
  'a',
  'comic',
  'book',
  'like',
  'from',
  'hell',
  'before',
  '.',
  'for',
  'starters',
  ',',
  'it',
  'was',
  'created',
  'by',
  'alan',
  'moore',
  '(',
  'and',
  'eddie',
  'campbell',
  ')',
  ',',
  'who',
  'brought',
  'the',
  'medium',
  'to',
  'a',
  'whole',
  'new',
  'level',
  'in',
  'the',
  'mid',
  "'",
  '80s',
  'with',
  'a',
  '12',
  '-',
  'part',
  'series',
  'called',
  'the',
  'watchmen',
  '.',
  'to',
  'say',
  'moore',
  'and',
  'campbell',
  'thoroughly',
  'researche

We separately split subjective and objective instances to keep a balanced uniform class distribution in both train and test sets.

In [6]:
# TODO: split training and testing data as 80/20
train_pos_docs = pos_docs[:int(len(pos_docs) * 0.8)]
test_pos_docs = pos_docs[int(len(pos_docs) * 0.8):]
train_neg_docs = neg_docs[:int(len(neg_docs) * 0.8)]
test_neg_docs = neg_docs[int(len(neg_docs) * 0.8):]

training_docs = train_pos_docs + train_neg_docs
testing_docs = test_pos_docs + test_neg_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

all_words_neg[:5]

['films', 'adapted', 'from', 'comic', 'books']

We use simple unigram word features, handling negation:

In [10]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
print(len(unigram_feats))
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

14667


We apply features to obtain a feature-value representation of our datasets:

In [8]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
print(training_set[0])

({'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, "contains(')": True, 'contains(to)': True, 'contains(is)': True, 'contains(in)': True, 'contains(s)': True, 'contains(")': True, 'contains(it)': True, 'contains(that)': True, 'contains(-)': True, 'contains(()': True, 'contains())': True, 'contains(as)': True, 'contains(with)': True, 'contains(his)': False, 'contains(for)': True, 'contains(this)': True, 'contains(film)': True, 'contains(i)': True, 'contains(he)': True, 'contains(but)': True, 'contains(on)': True, 'contains(are)': True, 'contains(t)': True, 'contains(the_NEG)': False, 'contains(,_NEG)': False, 'contains(by)': True, 'contains(be)': True, 'contains(who)': True, 'contains(an)': True, 'contains(movie)': False, 'contains(one)': True, 'contains(not)': True, 'contains(you)': True, 'contains(was)': True, 'contains(from)': True, 'contains(at)': True, 'contains(has)': True, 'contains(they)': True, 'c

We can now train our classifier on the training set, and subsequently output the evaluation results:

In [11]:
# TODO: Use Naive Bayes to train the sentiment classifier
naive_bayes = NaiveBayesClassifier.train
# svm = SklearnClassifier(LinearSVC()).train
# maxent = MaxentClassifier.train

classifier = sentim_analyzer.train(naive_bayes, training_set)

sentim_analyzer.evaluate(test_set).items()

Training classifier
Evaluating NaiveBayesClassifier results...


dict_items([('Accuracy', 0.8125), ('Precision [neg]', 0.7729257641921398), ('Recall [neg]', 0.885), ('F-measure [neg]', 0.8251748251748252), ('Precision [pos]', 0.8654970760233918), ('Recall [pos]', 0.74), ('F-measure [pos]', 0.797843665768194)])

### 2.2. Lexical approach

Lexical approaches aim to map words to sentiment by building a lexicon or a 'dictionary of sentiment'. We can use this dictionary to assess the sentiment of phrases and sentences, without the need of looking at anything else. Sentiment can be categorical – such as {negative, neutral, positive} – or it can be numerical – like a range of intensities or scores. Lexical approaches look at the sentiment category or score of each word in the sentence and decide what the sentiment category or score of the whole sentence is. The power of lexical approaches lies in the fact that we do not need to train a model using labeled data, since we have everything we need to assess the sentiment of sentences in the dictionary of emotions. VADER is an example of a lexical method.

In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [14]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...


True

Run the lexical approach

In [15]:
sid = SentimentIntensityAnalyzer()
for doc in testing_docs:
    doc = " ".join(doc[0])
    print(doc[:100] + "...")
    ss = sid.polarity_scores(doc)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

those of you who frequently read my reviews are not likely to be surprised by the fact that i have n...
compound: 0.9948, neg: 0.041, neu: 0.813, pos: 0.146, 
this is a stagy film adapted from roger rueff ' s play " hospitality suite . '' its about three trav...
compound: -0.3452, neg: 0.088, neu: 0.825, pos: 0.087, 
the coen brothers are back again , this time with homer ' s " odyssey " as the backdrop in their tal...
compound: 0.9975, neg: 0.067, neu: 0.795, pos: 0.138, 
contact ( pg ) there ' s a moment late in robert zemeckis ' s contact where i was reminded of why i ...
compound: 0.9989, neg: 0.053, neu: 0.776, pos: 0.171, 
satirical films usually fall into one of two categories : 1 ) long - term satire where everything , ...
compound: 0.9977, neg: 0.06, neu: 0.783, pos: 0.156, 
after sixteen years francis ford copolla has again returned to his favorite project , making the thi...
compound: 0.9913, neg: 0.131, neu: 0.695, pos: 0.174, 
bruce willis is a type - casted actor . in die

compound: 0.9871, neg: 0.088, neu: 0.783, pos: 0.128, 
the verdict : spine - chilling drama from horror maestro stephen king , featuring an outstanding , o...
compound: -0.9437, neg: 0.154, neu: 0.702, pos: 0.143, 
an indian runner was more than a courier . he * became * the message he was carrying . what danger i...
compound: -0.6161, neg: 0.109, neu: 0.794, pos: 0.097, 
seen july 8 , 1998 at the crossgates cinema 18 , ( albany , ny ) , theater # 7 , at 8 : 15 p . m . w...
compound: 0.999, neg: 0.099, neu: 0.733, pos: 0.168, 
not since attending an ingmar bergman retrospective a few years ago have i seen a film as uncompromi...
compound: 0.9914, neg: 0.066, neu: 0.797, pos: 0.137, 
after a successful run in australia last year , and with much critical praise heaped upon it , murie...
compound: 0.9941, neg: 0.1, neu: 0.695, pos: 0.205, 
every once in a while , a film sneaks up on me and takes me completely by surprise . i don ' t neces...
compound: 0.9986, neg: 0.064, neu: 0.752, pos: 

compound: 0.9964, neg: 0.059, neu: 0.776, pos: 0.165, 
if there is one thing that bothers me about hollywood films it ' s their predictable endings . the d...
compound: -0.4945, neg: 0.148, neu: 0.697, pos: 0.155, 
ingredients : down - on - his - luck evangelist , church synopsis : sonny dewey ( robert duvall ) is...
compound: 0.9901, neg: 0.065, neu: 0.801, pos: 0.134, 
the most amazing thing about paul cox ' s innocence is how unlike a movie it is . i mean that as the...
compound: 0.9966, neg: 0.05, neu: 0.787, pos: 0.164, 
contact is a film that tries to do several different things . it is intended to present a realistic ...
compound: 0.997, neg: 0.041, neu: 0.827, pos: 0.132, 
billy bob thornton , who had a sudden rise to fame with 1996 ' s sling blade after spending years as...
compound: -0.9445, neg: 0.172, neu: 0.67, pos: 0.158, 
tibet has entered the american consciousness slowly during the past few years and burst into the for...
compound: 0.9413, neg: 0.07, neu: 0.848, pos: 0

compound: -0.3034, neg: 0.133, neu: 0.731, pos: 0.136, 
i rented this movie with very high hopes . this movie got praise as one of the best films of 1998 , ...
compound: 0.998, neg: 0.07, neu: 0.745, pos: 0.185, 
few films in 1999 have divided the critical consensus as sharply as alan parker ' s adaptation of fr...
compound: 0.9918, neg: 0.101, neu: 0.751, pos: 0.148, 
now that " boogie nights " has made disco respectable again ( well , fashionable at least ) , we sho...
compound: 0.9981, neg: 0.026, neu: 0.784, pos: 0.19, 
this has been an extraordinary year for australian films . " shine " has just scooped the pool at th...
compound: 0.9956, neg: 0.066, neu: 0.782, pos: 0.152, 
i think the first thing this reviewer should mention is wether or not i am a fan of the x - files . ...
compound: 0.9985, neg: 0.078, neu: 0.777, pos: 0.145, 
trees lounge is the directoral debut from one of my favorite actors , steve buscemi . he gave memora...
compound: 0.9748, neg: 0.077, neu: 0.749, pos: 0

compound: -0.9172, neg: 0.114, neu: 0.799, pos: 0.087, 
" meg ryan is irresistible in the comedy that celebrates sisterhood ! , " screams the television ads...
compound: 0.9956, neg: 0.076, neu: 0.8, pos: 0.123, 
under any other circumstances , i would not be discussing the ending of a film to the extent that i ...
compound: -0.8821, neg: 0.113, neu: 0.796, pos: 0.092, 
brian de palma ' s snake eyes stars nicolas cage ' s evil twin , who confusingly uses the same stage...
compound: -0.8216, neg: 0.109, neu: 0.797, pos: 0.094, 
a disappointing biography about the homosexual relationship of two famous 19th century french poets ...
compound: -0.9839, neg: 0.136, neu: 0.763, pos: 0.1, 
in our time . in our modern world , where the cool rule , it ' s hard to imagine that shakespeare is...
compound: 0.9944, neg: 0.125, neu: 0.698, pos: 0.177, 
isn ' t it the ultimate sign of a movie ' s cinematic ineptitude when you can ' t think of much to s...
compound: 0.9522, neg: 0.124, neu: 0.728, pos:

compound: 0.7236, neg: 0.138, neu: 0.719, pos: 0.143, 
some movies ' pre - release buzz is so insistent on their high oscar potential that when they ' re f...
compound: 0.9719, neg: 0.099, neu: 0.771, pos: 0.13, 
all feature film directors who cut their teeth on music videos , please raise your hands . thank you...
compound: 0.988, neg: 0.083, neu: 0.799, pos: 0.117, 
depending on your degree of cinematic acumen , last man standing is either a ) a prohibition - era r...
compound: 0.9532, neg: 0.097, neu: 0.789, pos: 0.114, 
okay , bear with me y ' all , cause first off i have to get this off my chest : what the hell is up ...
compound: 0.9924, neg: 0.104, neu: 0.741, pos: 0.155, 
i went to blair witch project 2 : book of shadows with the highest of hopes . the original film , re...
compound: -0.9995, neg: 0.17, neu: 0.746, pos: 0.084, 
i ' m giving this stinker . normally , the worst that i would ever rate a movie would be . but the t...
compound: -0.9772, neg: 0.146, neu: 0.737, pos: 

compound: -0.9827, neg: 0.149, neu: 0.744, pos: 0.108, 
summer movies are , by nature , dumb affairs that are usually made for some quick enjoyment and to m...
compound: 0.9588, neg: 0.085, neu: 0.795, pos: 0.12, 
what were they thinking ? nostalgia for the seventies is bad enough , but do we really need an eight...
compound: 0.9839, neg: 0.065, neu: 0.789, pos: 0.146, 
and just when you thought joblo was getting a little soft around the corners , not rating anything l...
compound: 0.9816, neg: 0.091, neu: 0.789, pos: 0.12, 
` bats ' is an insulting slap across the face for any dedicated horror movie fan . to pull something...
compound: -0.9846, neg: 0.163, neu: 0.697, pos: 0.141, 
the most interesting part of " can ' t hardly wait " just happens to be not only the most human , bu...
compound: 0.9862, neg: 0.108, neu: 0.767, pos: 0.125, 
the film may be called mercury rising , but that title doesn ' t describe the trajectory taken by th...
compound: -0.9425, neg: 0.142, neu: 0.724, pos

### 2.3 Comparing two approaches

First we can transform the sentiment score by the lexical approach into label by the following rules:

+ positive sentiment: compound score > 0
+ negative sentiment: compound score <= 0

In [17]:
def lexical_sentiment(doc, sid=None):
    """TODO: return the label 'pos' or 'neg' for a document"""
    if sid is None:
        sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(doc)
    if ss["compound"] > 0:
        return 'pos'
    return 'neg'

for doc in testing_docs:
    doc = " ".join(doc[0])
    label = lexical_sentiment(doc, sid)
    print(doc[:100] + "...", label)

those of you who frequently read my reviews are not likely to be surprised by the fact that i have n... pos
this is a stagy film adapted from roger rueff ' s play " hospitality suite . '' its about three trav... neg
the coen brothers are back again , this time with homer ' s " odyssey " as the backdrop in their tal... pos
contact ( pg ) there ' s a moment late in robert zemeckis ' s contact where i was reminded of why i ... pos
satirical films usually fall into one of two categories : 1 ) long - term satire where everything , ... pos
after sixteen years francis ford copolla has again returned to his favorite project , making the thi... pos
bruce willis is a type - casted actor . in die hard , he played john mcclaine , a rough and tough ch... pos
there must be some unwritten rule that states , one gets enlightenment not in the way one expects to... pos
did claus von bulow try to kill his wife sunny in their newport mansion ? that is the question rever... neg
driving miss daisy takes its

in _daylight_ , sylvester stallone breaks no new ground , cinematically speaking , but he covers fam... neg
perhaps the most dramatic changes in the motion picture industry in this decade have to do with spec... pos
let me open this one with a confession : i love cop movies . i adore them with such an unwavering , ... pos
carla gugino graduates from high school and instead of staying in her small farming town , she goes ... pos
the soldiers of three kings have taken their cue from movies about vietnam . ( fitting , since the m... neg
bill condon ' s " gods and monsters " is a fascinating look into the last days in the life of gay di... pos
i must admit that i was a tad skeptical of " good will hunting " , based both on the previews and th... pos
a cinematic version of one of john irving ' s novels is always cause for celebration even if , as in... pos
the only historical figure that has been written about more than william shakespeare is jesus christ... pos
in 1912 , a ship set sail on

martin scorsese ' s films used to intimidate me . because of his reputation , i felt obligated to ap... pos
robert redford ' s a river runs through it is not a film i watch often . it is a masterpiece -- one ... pos
richard gere is not one of my favorite actors . however , i like courtroom dramas , and this film lo... pos
" when you get out of jail , you can kill him . " starring ashley judd , tommy lee jones , bruce gre... neg
let ' s say you live at the end of an airport runway . large jetliners continuously pass over your h... pos
in this good natured , pleasent and easy going comedy , bill murray ( ghostbusters , 1984 ) plays gr... pos
" no man is an island , " one character quotes john donne in apt pupil , effectively summarizing the... neg
i rented this movie with very high hopes . this movie got praise as one of the best films of 1998 , ... pos
few films in 1999 have divided the critical consensus as sharply as alan parker ' s adaptation of fr... pos
now that " boogie nights " h

burnt money is the perfect festival film . it will show once or twice , and then no one , thankfully... neg
claire danes , giovanni ribisi , and omar epps make a likable trio of protagonists , but they ' re j... pos
the haunting , a film so confusing that it forgets the true meaning of a horror film : to scare us .... neg
in 1989 , director edward zwick began his career with the powerful civil war drama , " glory , " but... neg
i guess that if a very wild bachelor party had gone really bad , there would be broken furniture , t... neg
i had been looking forward to this film since i heard about it early last year , when matthew perry ... pos
the original babe gets my vote as the best family film since the princess bride , and it ' s sequel ... pos
to put it bluntly , ed wood would have been proud of this . a totally ridiculous plot is encompassed... neg
i wonder if budget is at all a criterion for whether or not a movie can be considered an exploitatio... neg
one of these days , i ' ll m

this movie stinks ! although it is professionally crafted and there are some decent performances , t... neg
" it was not scary . " these are the first words that came to mind after it was over . when a movie ... neg
perhaps best remembered as the recently departed news anchor on saturday night live who always start... neg
summer movies are , by nature , dumb affairs that are usually made for some quick enjoyment and to m... pos
what were they thinking ? nostalgia for the seventies is bad enough , but do we really need an eight... pos
and just when you thought joblo was getting a little soft around the corners , not rating anything l... pos
` bats ' is an insulting slap across the face for any dedicated horror movie fan . to pull something... neg
the most interesting part of " can ' t hardly wait " just happens to be not only the most human , bu... pos
the film may be called mercury rising , but that title doesn ' t describe the trajectory taken by th... neg
" alcohol and drugs = bad . 

Now we evaluate the lexical approach by computing accuracy metrics

In [18]:
from collections import defaultdict
from nltk.metrics import (accuracy as eval_accuracy, precision as eval_precision,
        recall as eval_recall, f_measure as eval_f_measure)

gold_results = defaultdict(set)
test_results = defaultdict(set)
acc_gold_results = []
acc_test_results = []
labels = set()
num = 0
for i, (text, label) in enumerate(testing_docs):
    labels.add(label)
    gold_results[label].add(i)
    acc_gold_results.append(label)
    observed = lexical_sentiment(" ".join(text), sid)
    num += 1
    acc_test_results.append(observed)
    test_results[observed].add(i)
metrics_results = {}

# TODO: compute the accuracy metrics
metrics_results["Accuracy"] = eval_accuracy(acc_gold_results, acc_test_results)
for label in labels:
    metrics_results["F-measure [" + label + "]"] = eval_f_measure(gold_results[label], test_results[label])
    metrics_results["Precision [" + label + "]"] = eval_precision(gold_results[label], test_results[label])
    metrics_results["Recall [" + label + "]"] = eval_recall(gold_results[label], test_results[label])

for result in sorted(metrics_results):
        print('{0}: {1}'.format(result, metrics_results[result]))

Accuracy: 0.635
F-measure [neg]: 0.546583850931677
F-measure [pos]: 0.6945606694560669
Precision [neg]: 0.7213114754098361
Precision [pos]: 0.5971223021582733
Recall [neg]: 0.44
Recall [pos]: 0.83
